# Chapter 3. word2vec

## 3.1 추론 기반 기법과 신경망


### 3.1.1 통계 기반 기법의 문제점

대규모 말뭉치를 다룰 때 문제가 발생  
단 1회의 처리 (SVD 등)에 단어의 분산 표현: 배치 학습  
신경망이 한번에 소량 (미니배치)의 학습 샘플씩 반복해서 학습하며 가중치를 갱신  
게다가 여러 머신과 여러 GPU를 이용한 병렬 계산도 가능해져 학습 속도를 높일 수 있음  

*reference* 3.5.3 통계 기반 vs 추론 기반

### 3.1.2 추론 기반 기법 개요

모델로 신경망을 사용  
모델은 맥락 정보를 입력받아 (출현할 수 있는) 각 단어의 출현 확률을 출력  
학습의 결과로 단어의 분산 표현을 얻는 것  


### 3.1.3 신경망에서의 단어 처리

고정 길이의 벡터로 변환 - 원핫 표현  


In [2]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

c = np.array([[1, 0, 0, 0, 0, 0, 0]])
W = np.random.randn(7, 3)
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.48875471  0.02173541  1.08132014]]


## 3.2 단순한 word2vec

### 3.2.1 CBOW 모델의 추론 처리

사용할 신경망은 word2vec에서 제안하는 CBOW (continuous bag-of-words) 모델  
맥락 (주변 단어들)으로부터 타깃 (중앙 단어)을 추측하는 용도의 신경망  
입력: 맥락의 원핫 표현, 맥락으로 고려할 단어 2개로 정하면 입력층도 2이다.  
은닉: 입력층이 여러 개이면 전체를 '평균'  
출력: 뉴런 하나하나가 각각의 단어에 대응, 출력층 뉴런은 각 단어의 '점수'를 뜻하며, 소프트맥스 함수를 적용해서 '확률'을 얻음

은닉층의 뉴런 수를 입력층의 뉴런 수보다 적게 - 인코딩에 해당  
활성화 함수를 사용하지 않는 간단한 구성의 신경망  

* ch03/cbow_predict.py에 구현

### 3.2.2 CBOW 모델의 학습

확률이란 맥락 (전후 단어)이 주어졌을 때 그 중앙에 어떤 단어가 출현하는지  
정답에 해당하는 뉴런의 값이 클 것이라 기대할 수 있다.  
CBOW 모델의 학습에서 올바른 예측을 할 수 있도록 가중치를 조정하는 일을 한다.  
그러므로, 이 신경망을 학습하려면 소프트맥스와 교차 엔트로피 오차만 이용하면 된다.  

### 3.2.3 word2vec의 가중치와 분산 표현

두 가지 가중치: 입력 측 완전연결계층의 가중치 ($\mathbf{W}_{in}$), 출력 측 완전연결계층의 가중치($\mathbf{W}_{out}$)
* $\mathbf{W}_{in}$ - 각 행이 각 단어의 분산 표현
* $\mathbf{W}_{out}$ - 단어의 의미가 인코딩된 벡터가 저장


## 3.3 학습 데이터 준비

* 입력: 맥락의 각 행
* 타깃: 각 행이 정답 레이블

In [6]:
# 말뭉치 텍스트를 단어 ID로 변환 (2장에서 구현한 preprocess 함수 사용)

import sys
sys.path.append('..')
from common.util import preprocess, create_contexts_target, convert_one_hot

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
print(corpus, id_to_word)

[0 1 2 3 4 1 5 6] {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


* 맥락과 타깃을 만드는 함수를 구현
* common/util.py에 create_contexts_target 함수로 구현

In [4]:
contexts, target = create_contexts_target(corpus, window_size=1)
print(contexts, target)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]] [1 2 3 4 1 5]


In [7]:
vocab_size = len(word_to_id)
target = convert_one_hot(target, vocab_size)
contexts = convert_one_hot(contexts, vocab_size)

## 3.4 CBOW 모델 구현

* ch03/simple_cbow.py에 구현